In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
X = tf.range(10)
for item in X:
    print(item)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
def sqrt(x):
    return x ** 2

In [4]:
# dataset = dataset.map(lambda x: sqrt(x))
# dataset = dataset.map(lambda x: x ** 2)
dataset = dataset.map(sqrt)
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(16, shape=(), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(36, shape=(), dtype=int32)
tf.Tensor(49, shape=(), dtype=int32)
tf.Tensor(64, shape=(), dtype=int32)
tf.Tensor(81, shape=(), dtype=int32)


In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

In [6]:
dataset = fetch_california_housing()
X, y = dataset.data, dataset.target.reshape(-1, 1)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

In [7]:
def split_dataset_to_csv(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join("datasets", "housing")
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, "my_{}_{:02d}.scv")
    
    file_paths = []
    
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        file_paths.append(part_csv)
        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header)
                f.write("\n")
            for row_idx in row_indices:
                f.write(','.join([repr(col) for col in data[row_idx]]))
                f.write("\n")
        
    return file_paths

In [8]:
train_data = np.c_[X_train, y_train]
test_data = np.c_[X_test, y_test]
val_data = np.c_[X_val, y_val]
header_cols = dataset.feature_names + ["MedianHouseValue"]
header = ','.join(header_cols)

train_file_paths = split_dataset_to_csv(train_data, "train", header, n_parts=20)
valid_file_paths = split_dataset_to_csv(val_data, "valid", header)
test_file_paths = split_dataset_to_csv(test_data, "test", header)

In [9]:
import pandas as pd

In [10]:
pd.read_csv(train_file_paths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,3.5214,15.0,3.049945,1.106548,1447.0,1.605993,37.63,-122.43,1.442
1,5.3275,5.0,6.490060,0.991054,3464.0,3.443340,33.69,-117.39,1.687
2,3.1000,29.0,7.542373,1.591525,1328.0,2.250847,38.44,-122.98,1.621
3,7.1736,12.0,6.289003,0.997442,1054.0,2.695652,33.55,-117.70,2.621
4,2.0549,13.0,5.312457,1.085092,3297.0,2.244384,33.93,-116.93,0.956


In [11]:
train_file_paths[0]

'datasets\\housing\\my_train_00.scv'

In [12]:
filepath_dataset = tf.data.Dataset.list_files(train_file_paths)
for file in filepath_dataset:
    print(file)

tf.Tensor(b'datasets\\housing\\my_train_04.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_01.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_14.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_05.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_10.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_11.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_19.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_07.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_18.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_15.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_12.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_09.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_06.scv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_13.scv', sh

In [13]:
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=5)

In [14]:
for line in dataset.take(5):
    print(line)

tf.Tensor(b'2.3,25.0,5.828178694158075,0.9587628865979382,909.0,3.1237113402061856,36.25,-119.4,1.328', shape=(), dtype=string)
tf.Tensor(b'2.4792,24.0,3.4547038327526134,1.1341463414634145,2251.0,3.921602787456446,34.18,-118.38,2.0', shape=(), dtype=string)
tf.Tensor(b'4.1812,52.0,5.701388888888889,0.9965277777777778,692.0,2.4027777777777777,33.73,-118.31,3.215', shape=(), dtype=string)
tf.Tensor(b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418', shape=(), dtype=string)
tf.Tensor(b'4.2708,45.0,5.121387283236994,0.953757225433526,492.0,2.8439306358381504,37.48,-122.19,2.67', shape=(), dtype=string)


In [15]:
n_inputs = 8
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y
preprocess(b'4.5909,16.0,5.475877192982456,1.0964912280701755,1357.0,2.9758771929824563,33.63,-117.71,2.418')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.36618188, -0.998705  ,  0.00781878, -0.00675364, -0.06140145,
         0.0072037 , -0.94465536,  0.9367464 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.418], dtype=float32)>)

In [16]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [17]:
train_set = csv_reader_dataset(train_file_paths, repeat=None)
test_set = csv_reader_dataset(test_file_paths)
val_set = csv_reader_dataset(valid_file_paths)

In [18]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=[8,]),
    keras.layers.Dense(1)
    
])

In [19]:
print(len(X_train))
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))
batch_size = 32
model.fit(train_set, steps_per_epoch=len(X_train) // batch_size, epochs=10,
          validation_data=val_set)

11610
Epoch 1/10
362/362 [==============================] - 2s 4ms/step - loss: 1.7350 - val_loss: 1.0619
Epoch 2/10
362/362 [==============================] - 0s 1ms/step - loss: 0.7523 - val_loss: 0.8050
Epoch 3/10
362/362 [==============================] - 0s 1ms/step - loss: 0.6977 - val_loss: 0.6797
Epoch 4/10
362/362 [==============================] - 0s 1ms/step - loss: 0.6493 - val_loss: 0.6053
Epoch 5/10
362/362 [==============================] - 0s 1ms/step - loss: 0.6004 - val_loss: 0.6088
Epoch 6/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5873 - val_loss: 0.5957
Epoch 7/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5642 - val_loss: 0.5474
Epoch 8/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5176 - val_loss: 0.5032
Epoch 9/10
362/362 [==============================] - 0s 1ms/step - loss: 0.5202 - val_loss: 0.5099
Epoch 10/10
362/362 [==============================] - 0s 1ms/step - loss: 0.4980 - val_loss: 

# TFRecord

In [20]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"First Record")
    f.write(b"Second Record")

In [21]:
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'First Record', shape=(), dtype=string)
tf.Tensor(b'Second Record', shape=(), dtype=string)


# Compressing TFRecord Files 

In [22]:
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"Babingobingo")
    f.write(b"Zingobenzingo")
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"], compression_type="GZIP")

In [23]:
for file in dataset:
    print(file)

tf.Tensor(b'Babingobingo', shape=(), dtype=string)
tf.Tensor(b'Zingobenzingo', shape=(), dtype=string)


# Protocol Buffers

In [24]:
%%writefile person.proto
syntax = "proto3";
message Person {
  string name = 1;
  int32 id = 2;
  repeated string email = 3;
}

Overwriting person.proto


In [25]:
!protoc person.proto --python_out=. --descriptor_set_out=person.desc --include_imports

In [26]:
!ls person*

person.desc
person.proto
person_pb2.py


In [27]:
from person_pb2 import Person

person = Person(name="al", id=2, email=["a@b.com"])

In [28]:
print(person)

name: "al"
id: 2
email: "a@b.com"

